In [1]:
from __future__ import print_function
import sys
sys.path.append('../')
sys.path.append('/')

from argparse import ArgumentParser
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import random
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from tqdm import tqdm
import numpy as np
import pdb
# from torch.utils.tensorboard import SummaryWriter
from glob import glob
import pandas as pd
from metrics_manager import metrics_manager
import time
import wandb
from collections import OrderedDict
import random
from BigredDataSet import BigredDataSet
from BigredDataSetPTG import BigredDataSetPTG
from opt_deepgcn import OptInit as OptInit_deepgcn
from torch_geometric.data import DenseDataLoader
import torch_geometric.transforms
from torch.nn import Sequential as Seq
from kornia.utils.metrics import mean_iou,confusion_matrix
import pandas as pd
import importlib

def convert_state_dict(state_dict):
    if not next(iter(state_dict)).startswith("module."):
        return state_dict  # abort if dict is not a DataParallel model_state
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:]  # remove `module.`
        new_state_dict[name] = v
    return new_state_dict

In [17]:
import torchvision.models as models
import torch
from ptflops import get_model_complexity_info
load_pretrain = 'ckpt/pointnetpp_4c_single'
pretrained_model_path = os.path.join(load_pretrain,'saves/best_model.pth')
package = torch.load(pretrained_model_path)
para_state_dict = package['state_dict']
num_channel = package['num_channel']

time = package['time'] 
epoch_ckpt = package['epoch']

state_dict = convert_state_dict(para_state_dict)

ckpt_,ckpt_file_name  = load_pretrain.split("/")
module_name = ckpt_+'.'+ckpt_file_name+'.'+'model'
MODEL = importlib.import_module(module_name)
model = MODEL.get_model(input_channel = num_channel)


with torch.cuda.device(0):
    macs, params = get_model_complexity_info(model, (20000, 4), as_strings=True,print_per_layer_stat=True, verbose=True)
    print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
    print('{:<30}  {:<8}'.format('Number of parameters: ', params))

ValueError: SyncBatchNorm expected input tensor to be on GPU